In [1]:
!nvidia-smi

Sat Oct  7 16:21:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q pandas

In [3]:
!pip install -q autotrain-advanced

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 45.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 53.0 MB/s eta 0

In [4]:
!autotrain setup --update-torch

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from datasets import load_dataset

#Loading Medical Dataset
dataset = load_dataset("BI55/MedText")

print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Prompt', 'Completion'],
        num_rows: 1412
    })
})


In [7]:
import pandas as pd
medical_data=pd.DataFrame(dataset['train'])
medical_data.head()

,Prompt,Completion
0,A 50-year-old male presents with a history of ...,This patient's history of recurrent kidney sto...
1,"A 7-year-old boy presents with a fever, headac...","This child's symptoms of a red, bulging tympan..."
2,A 35-year-old woman presents with a persistent...,While the symptoms might initially suggest ast...
3,A 50-year-old male presents with severe abdomi...,The patient's symptoms suggest an incarcerated...
4,A newborn baby presents with eye redness and a...,The infant's symptoms suggest neonatal conjunc...


In [8]:
# Converting data to model aligned fine-tuning

medical_data['text']="###Human:\n"+medical_data['Prompt']+"\n\n"+"###Assistant:\n"+medical_data['Completion']
medical_data

,Prompt,Completion,text
0,A 50-year-old male presents with a history of ...,This patient's history of recurrent kidney sto...,###Human:\nA 50-year-old male presents with a ...
1,"A 7-year-old boy presents with a fever, headac...","This child's symptoms of a red, bulging tympan...",###Human:\nA 7-year-old boy presents with a fe...
2,A 35-year-old woman presents with a persistent...,While the symptoms might initially suggest ast...,###Human:\nA 35-year-old woman presents with a...
3,A 50-year-old male presents with severe abdomi...,The patient's symptoms suggest an incarcerated...,###Human:\nA 50-year-old male presents with se...
4,A newborn baby presents with eye redness and a...,The infant's symptoms suggest neonatal conjunc...,###Human:\nA newborn baby presents with eye re...
...,...,...,...
1407,A 55-year-old male with a history of chronic o...,While this patient's symptoms could be due to ...,###Human:\nA 55-year-old male with a history o...
1408,Can diet and lifestyle changes help manage vit...,While there is no specific diet or lifestyle m...,###Human:\nCan diet and lifestyle changes help...
1409,A 50-year-old female presents with right shoul...,This patient's shoulder and arm pain following...,###Human:\nA 50-year-old female presents with ...
1410,A 60-year-old female with high cholesterol lev...,In addition to a diet low in saturated fats an...,###Human:\nA 60-year-old female with high chol...


In [9]:
medical_data['text'][369]

'###Human:\nA 45-year-old female patient presents with recurring inflammatory flare-ups of osteoarthritis in her knees, causing discomfort and limited mobility. She prefers to avoid pharmaceutical options if possible. What might you suggest?\n\n###Assistant:\nGiven her preference for non-pharmaceutical options, a dietary supplement like turmeric might be beneficial. Curcumin, the active ingredient in turmeric, has been shown to have anti-inflammatory and analgesic properties, which can help reduce the inflammation and pain associated with osteoarthritis.'

In [10]:
medical_data.to_csv('train.csv',index=False)

In [ ]:
!autotrain llm --train --project_name Buffer --model mistralai/Mistral-7B-v0.1 --data_path . --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 4 --num_train_epochs 3 --trainer sft --target_modules q_proj,v_proj --push_to_hub --repo_id guna999/Buffer

Traceback (most recent call last):
  File "/usr/local/bin/autotrain", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/autotrain/cli/autotrain.py", line 46, in main
    command.run()
  File "/usr/local/lib/python3.10/dist-packages/autotrain/cli/run_llm.py", line 423, in run
    from autotrain.trainers.clm.__main__ import train as train_llm
  File "/usr/local/lib/python3.10/dist-packages/autotrain/trainers/clm/__main__.py", line 23, in <module>
    from trl import SFTTrainer
ModuleNotFoundError: No module named 'trl'


In [11]:
!pip install -q autotrain-advanced
!autotrain setup --update-torch

!autotrain llm -h

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch
usage: autotrain <command> [<args>] llm
       [-h]
       [--train]
       [--deploy]
       [--inference]
       [--data_path DATA_PATH]
       [--train_split TRAIN_SPLIT]
       [--valid_split VALID_SPLIT]
       [--text_column TEXT_COLUMN]
       [--model MODEL]
       [--learning_rate LEARNING_RATE]
       [--num_train_epochs NUM_TRAIN_EPOCHS]
       [--train_batch_size TRAIN_BATCH_SIZE]
       [--warmup_ratio WARMUP_RATIO]
       [--gradient_accumulation_steps GRA

In [12]:
!pip install -q safetensors

In [13]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers

# model_name = "bn22/Mistral-7B-Instruct-v0.1-sharded"
model_name="Naveen5/mistral-7b-medigpt-finetuned"
# adapters_name = "Naveen5/mistral-7b-medigpt-finetuned"

device = "cuda" # the device to load the model onto

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map='auto'
)
# model.config.use_cache = False
# model.config.use_ram_optimized_load=False

# model = PeftModel.from_pretrained(model, adapters_name,offload_folder="offload/")
# model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Successfully loaded the model Naveen5/mistral-7b-medigpt-finetuned into memory


In [14]:
def substitute(text):
  inp_text=text
  encoded = tokenizer(inp_text, return_tensors="pt", add_special_tokens=False)
  model_input = encoded
  # model.to(device)
  generated_ids = model.generate(**model_input, max_new_tokens=200, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

In [15]:
!pip install gradio
!pip install --upgrade gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.5.0
    Uninstalling gradio_client-0.5.0:
      Successfully uninstalled gradio_client-0.5.0
  Attempting uninstall: gradio
    Found existing installation: gradio 3.41.0
    Uninstalling gradio-3.41.0:
      Successfully uninstalled gradio-3.41.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.6.35 requires gradio==3.41.0, but you have gradio 3.47.1 which is incompatible.


In [16]:

import torch
from transformers import AutoTokenizer
import gradio as gr

model_name="Naveen5/mistral-7b-medigpt-finetuned"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
stop_token_ids = [0]

# def generate_response(text):
#     inp_text =str(text)
#     encoded = tokenizer(inp_text, return_tensors="pt", add_special_tokens=True)
#     model_input = encoded
#     # Here you would call your model to generate a response
#     # For the sake of this example, let's just return the input text
#     # output = inp_text
#     # return decoded[0]

iface = gr.Interface(fn=substitute, inputs="text", outputs="text")
iface.launch()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2fa64c4eb5a0ca8aba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
